# Exploratory notebook for miniblock decoding analysis

In [2]:
import numpy as np
import scipy.stats as stats
import h5py
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
import pandas as pd
import loadTaskBehavioralData as task
import tools
import multiprocessing as mp
import statsmodels.stats.multitest as mc
%matplotlib inline

#### Parameter set up

In [41]:
projectdir = '/projects3/CPROCompositionality/'
datadir = projectdir + 'data/processedData/' 
resultdir = projectdir + 'data/results/'
subjNums = ['013','014','016','017','018','021','023','024','026','027','028',
            '030','031','032','033','034','035','037','038','039','040','041',
            '042','043','045','046','047','048','049','050','053','055','056',
            '057','058','062','063','066','067','068','069','070','072','074',
            '075','076','077','081','085','086','087','088','090','092','093',
            '094','095','097','098','099','101','102','103','104','105','106',
            '108','109','110','111','112','114','115','117','119','120','121',
            '122','123','124','125','126','127','128','129','130','131','132',
            '134','135','136','137','138','139','140','141']

glasser = projectdir + 'data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser = nib.load(glasser).get_data()
glasser = np.squeeze(glasser)
rois = np.arange(1,361)

#### Load in CAB-NP ROI labels
cabn_labels = pd.read_csv(projectdir + 'data/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_LabelKey.txt',header=0,delimiter='\t')
df_labels = cabn_labels.iloc[0:360]
df_labels.reset_index()
del df_labels['INDEX'], df_labels["KEYVALUE"]

# Using final partition
networkdef = df_labels['NETWORKKEY'].values
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}

fpn_ind = np.where(networkdef==networkmappings['fpn'])[0]
con_ind = np.where(networkdef==networkmappings['con'])[0]
ccn_ind = np.hstack((fpn_ind,con_ind))

#### Load results for novelty decoding

In [48]:
df_novelty = pd.read_csv(resultdir + 'CrossSubjectNoveltyDecoding/CrossSubjectNoveltyDecoding_allROIs.csv')
rois = np.unique(df_novelty.ROI)
accuracy = []
ps = []
for roi in np.sort(rois):
    tmp_df = df_novelty.loc[df_novelty.ROI==roi]
    acc = tmp_df.DecodingAccuracy.values
    accuracy.append(np.mean(acc))

#### Load in accuracies from permutation test
nulldist = np.loadtxt(resultdir + 'CrossSubjectNoveltyDecoding/CrossSubjectNoveltyDecoding_NullDistribution.csv')
fwe_thresh = np.max(nulldist[ccn_ind,:],axis=0)
fwe_thresh = np.sort(fwe_thresh)
alpha_ind = int(len(fwe_thresh)*.95)
fwe_thresh = fwe_thresh[alpha_ind]

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[ccn_ind,1] = np.multiply(accuracy[ccn_ind],accuracy[ccn_ind]>fwe_thresh)
tools.mapBackToSurface(data,resultdir + 'NoveltyDecoding')

#### Identify which regions are sensitive to negations 

In [49]:
df_negation = pd.read_csv(resultdir + 'CrossSubjectLogicalNegationDecoding/CrossSubjectLogicalNegationDecoding_allROIs.csv')
rois = np.unique(df_negation.ROI)
accuracy = []
ps = []
for roi in np.sort(rois):
    tmp_df = df_negation.loc[df_negation.ROI==roi]
    acc = tmp_df.DecodingAccuracy.values
    accuracy.append(np.mean(acc))

#### Load in accuracies from permutation test
nulldist = np.loadtxt(resultdir + 'CrossSubjectLogicalNegationDecoding/CrossSubjectLogicalNegationDecoding_NullDistribution.csv')
fwe_thresh = np.max(nulldist[ccn_ind,:],axis=0)
fwe_thresh = np.sort(fwe_thresh)
alpha_ind = int(len(fwe_thresh)*.95)
fwe_thresh = fwe_thresh[alpha_ind]

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[ccn_ind,1] = np.multiply(accuracy[ccn_ind],accuracy[ccn_ind]>fwe_thresh)
tools.mapBackToSurface(data,resultdir + 'LogicalNegationDecoding')

#### Identify which regions are sensitive to task performance (correct v. error) 

In [50]:
df_performance = pd.read_csv(resultdir + 'CrossSubjectPerformanceDecoding/CrossSubjectPerformanceDecoding_allROIs.csv')
chance = 0.5
rois = np.unique(df_performance.ROI)
accuracy = []
ps = []
for roi in np.sort(rois):
    tmp_df = df_performance.loc[df_performance.ROI==roi]
    acc = tmp_df.DecodingAccuracy.values
    accuracy.append(np.mean(acc))

#### Load in accuracies from permutation test
nulldist = np.loadtxt(resultdir + 'CrossSubjectPerformanceDecoding/CrossSubjectPerformanceDecoding_NullDistribution.csv')
fwe_thresh = np.max(nulldist[ccn_ind,:],axis=0)
fwe_thresh = np.sort(fwe_thresh)
alpha_ind = int(len(fwe_thresh)*.95)
fwe_thresh = fwe_thresh[alpha_ind]

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[ccn_ind,1] = np.multiply(accuracy[ccn_ind],accuracy[ccn_ind]>fwe_thresh)
tools.mapBackToSurface(data,resultdir + 'PerformanceDecoding')

___

#### Load results for 64 task decoding

In [148]:
rois = np.arange(1,361)
chance = 1/64.0
accuracy = []
ps = []
for roi in np.sort(rois):
    df_64 = pd.read_csv(resultdir + 'CrossSubject64TaskDecoding/CrossSubject64TaskDecoding_roi' + str(roi) + '.csv')
    subjs = np.unique(df_64.Subject.values)
    subjaccuracy = []
    for subj in subjs:
        tmpdf = df_64.loc[df_64.Subject==subj]
        acc = np.mean(tmpdf.DecodingAccuracy.values)
        subjaccuracy.append(acc)
    t, p = stats.ttest_1samp(subjaccuracy,chance)
    p = p/2.0 if np.mean(subjaccuracy)>chance else 1.0-p/2.0
    ps.append(p)
    accuracy.append(np.mean(subjaccuracy))

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
sig_acc = np.multiply(accuracy,qs)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[:,1] = sig_acc
tools.mapBackToSurface(data,resultdir + '64TaskDecoding')

#### Load results for logic rule decoding

In [153]:
rois = np.arange(1,361)
chance = 1/4.0
accuracy = []
ps = []
for roi in np.sort(rois):
    df_logic = pd.read_csv(resultdir + 'CrossSubjectLogicRuleDecoding/CrossSubjectLogicRuleDecoding_roi' + str(roi) + '.csv')
    subjs = np.unique(df_logic.Subject.values)
    subjaccuracy = []
    for subj in subjs:
        tmpdf = df_logic.loc[df_logic.Subject==subj]
        acc = np.mean(tmpdf.DecodingAccuracy.values)
        subjaccuracy.append(acc)
    t, p = stats.ttest_1samp(subjaccuracy,chance)
    p = p/2.0 if np.mean(subjaccuracy)>chance else 1.0-p/2.0
    ps.append(p)
    accuracy.append(np.mean(subjaccuracy))

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
sig_acc = np.multiply(accuracy,qs)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[:,1] = sig_acc
tools.mapBackToSurface(data,resultdir + 'LogicRuleDecoding')

#### Load results for sensory rule decoding

In [154]:
rois = np.arange(1,361)
chance = 1/4.0
accuracy = []
ps = []
for roi in np.sort(rois):
    df_sensory = pd.read_csv(resultdir + 'CrossSubjectSensoryRuleDecoding/CrossSubjectSensoryRuleDecoding_roi' + str(roi) + '.csv')
    subjs = np.unique(df_sensory.Subject.values)
    subjaccuracy = []
    for subj in subjs:
        tmpdf = df_sensory.loc[df_logic.Subject==subj]
        acc = np.mean(tmpdf.DecodingAccuracy.values)
        subjaccuracy.append(acc)
    t, p = stats.ttest_1samp(subjaccuracy,chance)
    p = p/2.0 if np.mean(subjaccuracy)>chance else 1.0-p/2.0
    ps.append(p)
    accuracy.append(np.mean(subjaccuracy))

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
sig_acc = np.multiply(accuracy,qs)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[:,1] = sig_acc
tools.mapBackToSurface(data,resultdir + 'SensoryRuleDecoding')

#### Load results for motor rule decoding

In [155]:
rois = np.arange(1,361)
chance = 1/4.0
accuracy = []
ps = []
for roi in np.sort(rois):
    df_motor = pd.read_csv(resultdir + 'CrossSubjectMotorRuleDecoding/CrossSubjectMotorRuleDecoding_roi' + str(roi) + '.csv')
    subjs = np.unique(df_motor.Subject.values)
    subjaccuracy = []
    for subj in subjs:
        tmpdf = df_motor.loc[df_logic.Subject==subj]
        acc = np.mean(tmpdf.DecodingAccuracy.values)
        subjaccuracy.append(acc)
    t, p = stats.ttest_1samp(subjaccuracy,chance)
    p = p/2.0 if np.mean(subjaccuracy)>chance else 1.0-p/2.0
    ps.append(p)
    accuracy.append(np.mean(subjaccuracy))

qs = mc.fdrcorrection(ps)[0]
accuracy = np.asarray(accuracy)
sig_acc = np.multiply(accuracy,qs)
data = np.zeros((len(accuracy),2))
data[:,0] = accuracy
data[:,1] = sig_acc
tools.mapBackToSurface(data,resultdir + 'MotorRuleDecoding')